In [ ]:
# Deleting tables left from previous runs in case they still exist after deleting an inactive cluster
dbutils.fs.rm("/user", recurse=True)

Out[1]: True

In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Loading PySpark modules that we need
import unittest
from collections import Counter
from pyspark.sql import DataFrame
from pyspark.sql.types import *

#### Subtask 1: defining the schema for the data
Typically, the first thing to do before loading the data into a Spark cluster is to define the schema for the data. Look at the schema for 'badges' and try to define the schema for other tables similarly.

In [ ]:
# Defining a schema for 'badges' table
badges_schema = StructType([StructField('UserId', IntegerType(), False),
                            StructField('Name', StringType(), False),
                            StructField('Date', TimestampType(), False),
                            StructField('Class', IntegerType(), False)])

# Defining a schema for 'posts' table
posts_schema = StructType([StructField('Id', IntegerType(), False),
                           StructField('ParentId', IntegerType(), True),
                           StructField('PostTypeId', IntegerType(), False),
                           StructField('CreationDate', TimestampType(), False),
                            StructField('Score', IntegerType(), False),
                            StructField('ViewCount', IntegerType(), False),
                            StructField('Body', StringType(), False),
                            StructField('OwnerUserId', IntegerType(), False),
                            StructField('LastActivityDate', TimestampType(), False),
                            StructField('Title', StringType(), True),
                            StructField('Tags', StringType(), True),
                            StructField('AnswerCount', IntegerType(), False),
                            StructField('CommentCount', IntegerType(), False),
                            StructField('FavoriteCount', IntegerType(), False),
                            StructField('Closedate', TimestampType(), True)])
## YOUR IMPLEMENTATION ##

# Defining a schema for 'users' table
users_schema = StructType([StructField('Id', IntegerType(), False),
                           StructField('Reputation', IntegerType(), False),
                           StructField('CreationDate', TimestampType(), False),
                           StructField('DisplayName', StringType(), False),
                           StructField('LastAccessDate', TimestampType(), False),
                           StructField('AboutMe', StringType(), True),
                           StructField('Views', IntegerType(), False),
                           StructField('UpVotes', IntegerType(), False),
                           StructField('Downvotes', IntegerType(), False)])
## YOUR IMPLEMENTATION ##

# Defining a schema for 'comments' table
comments_schema = StructType([StructField('PostId', IntegerType(), False), 
                              StructField('Score', IntegerType(), False), 
                              StructField('Text', StringType(), False), 
                              StructField('CreationDate', TimestampType(), False), 
                              StructField('UserId', IntegerType(), False)])
## YOUR IMPLEMENTATION ##

#### Subtask 2: implementing two helper functions
Next, we need to implement two helper functions:
1. 'load_csv' that as input argument receives path for a CSV file and a schema and loads the CSV pointed by the path into a Spark DataFrame and returns the DataFrame;
2. 'save_df' receives a Spark DataFrame and saves it as a Parquet file on DBFS.

Note that the column separator in CSV files is TAB character ('\t') and the first row includes the name of the columns. 

BTW, DBFS is the name of the distributed filesystem used by Databricks Community Edition to store and access data.

In [ ]:
def load_csv(source_file: "path for the CSV file to load", schema: "schema for the CSV file being loaded as a DataFrame") -> DataFrame:
  # Since header is explicit from schema, we actively remove them from our imports
  return (spark.read
         .option("delimiter", '\t')
         .option("header", True)
         .schema(schema)
         .csv(source_file))

def save_df(df: "DataFrame to be saved", table_name: "name under which the DataFrame will be saved") -> None:
  df.write.format("parquet").save("/user/hive/warehouse/" + table_name)
  #df.write.csv("/user/hive/warehouse/" + table_name)

In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully. Moreover, at the end there should be four Parquet files named 'badges', 'comments', 'posts', and 'users' in '/user/hive/warehouse'.

Note that we assumed that the data for the project has already been stored on DBFS on the '/FileStore/tables/' path. (I mean as 'badges_csv.gz', 'comments_csv.gz', 'posts_csv.gz', and 'users_csv.gz'.)

In [ ]:
%%unittest_main
class TestTask1(unittest.TestCase):
   
    # test 1
    def test_load_badges(self):
        result = load_csv(source_file="/FileStore/tables/badges_csv.gz", schema=badges_schema)
        self.assertIsNotNone(result, "Badges dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 105640, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['UserId', 'Name', 'Date', 'Class']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 2
    def test_load_posts(self):
        result = load_csv(source_file="/FileStore/tables/posts_csv.gz", schema=posts_schema)
        self.assertIsNotNone(result, "Posts dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 61432, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'ParentId', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
                                    'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
                                    'CloseDate']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 3
    def test_load_comments(self):
        result = load_csv(source_file="/FileStore/tables/comments_csv.gz", schema=comments_schema)
        self.assertIsNotNone(result, "Comments dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 58735, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['PostId', 'Score', 'Text', 'CreationDate', 'UserId']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 4
    def test_load_users(self):
        result = load_csv(source_file="/FileStore/tables/users_csv.gz", schema=users_schema)
        self.assertIsNotNone(result, "Users dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 91616, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'AboutMe',
                                    'Views', 'UpVotes', 'DownVotes']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    # test 5
    def test_save_dfs(self):
        dfs = [("/FileStore/tables/users_csv.gz", users_schema, "users"),
               ("/FileStore/tables/badges_csv.gz", badges_schema, "badges"),
               ("/FileStore/tables/comments_csv.gz", comments_schema, "comments"),
               ("/FileStore/tables/posts_csv.gz", posts_schema, "posts")
               ]

        for i in dfs:
            df = load_csv(source_file=i[0], schema=i[1])
            save_df(df, i[2])

Success.....
----------------------------------------------------------------------
Ran 5 tests in 57.880s

OK
Out[7]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'Spark Application', 'SparkSession', 'Transformations', 'Action', and 'Lazy Evaluation' mean in the context of Spark?

Write your descriptions in the next cell.

**Spark Application** is, with the lack of a better term the core of Spark and describes the how the Driver process and Worker processes interact. The Worker processes, or excecutors, are nodes responsible for the actual computations. They are assigned a specific task that they must carry out and return the result back to the Driver. Note that the Workers can also cache, or presist a query result. The Driver process is essentially the `main()`-method of our application. The driver, based on a user's code or input, creates a plan consisting of what operations that has to be excecuted and schedules it to the workers/executors. 

**SparkSession** is also a building block of using Spark, which is called by the Spark Application. A SparkSession is necessary in order to work with the Spark objects `DataFrame`, `RDD` and `Dataset`. SparkSession was implemented in Spark 2, partially replacing the elder SparkContext class

***Transformations*** are one of the two main operations you can do with data in Spark. Transformations will change, or *transform* the data into something else based on some criteria, condition or function. A log-scaling of stock price data can be an example of a trasnformation, but filterings are also a type of transformation, so it does not have to be bijective. 

***Action*** is the other operation you can do on data, which evaluates to a specific result. Examples of actions can be `count`, `max`, `min`, `first`, `last`, `average` and so on. They often aggregate over the data to produce a certain result, and not necessarily preform any manipulations to the data. 

***Lazy Evaluation*** is an important concept in regards to the efficiency of Spark. This entails that the data will not be transformed before an action is actually called. Instead, Spark stores the call stack. Once the user needs the result from the transformation, i.e. the user has called an action on a transformed RDD, the transformation is preformed.